In [1]:
import pandas as pd
import numpy as np
from IPython.display import display
import random
from sklearn.model_selection import LeaveOneOut
from sklearn.linear_model import LinearRegression, LogisticRegression
import random
from sklearn.metrics import recall_score, confusion_matrix, accuracy_score
from sklearn.model_selection import train_test_split
from IPython.display import display

In [2]:
tagsFile = "tags"
recipesFile = "recipes"
nutritionsFile = "nutritions"
ingredientsFile = "ingredients"

tagsdf = pd.read_csv(f"/data/foodboost/{tagsFile}.csv", index_col=0)
recipesdf = pd.read_csv(f"/data/foodboost/{recipesFile}.csv", index_col=0)
nutritionsdf = pd.read_csv(f"/data/foodboost/{nutritionsFile}.csv", index_col=0)
ingredientsdf = pd.read_csv(f"/data/foodboost/{ingredientsFile}.csv", index_col=0)
mergedf = pd.read_csv("food_merge_2.csv", index_col=0)

recipesdf['RecipeId'] = range(1, len(recipesdf) + 1)
recipesdf['Rating'] = np.random.randint(1, 6, recipesdf.shape[0])
recipesdf.set_index('RecipeId')
recipesdf.head()

,title,persons,time,calories,stars,url,image,RecipeId,Rating
0,Kruidnoten met choco-discodip,4,25,260,0,https://www.ah.nl/allerhande/recept/R-R1195893...,https://static.ah.nl/static/recepten/img_RAM_P...,1,3
0,Kruidnoten in marsepein,4,25,265,0,https://www.ah.nl/allerhande/recept/R-R1195892...,https://static.ah.nl/static/recepten/img_RAM_P...,2,5
0,Kruidnoten met chocodips,4,25,335,0,https://www.ah.nl/allerhande/recept/R-R1195891...,https://static.ah.nl/static/recepten/img_RAM_P...,3,4
0,Pepernotentaart met marsepeinstrik,10,30,560,0,https://www.ah.nl/allerhande/recept/R-R1195887...,https://static.ah.nl/static/recepten/img_RAM_P...,4,4
0,Perencake,12,10,265,0,https://www.ah.nl/allerhande/recept/R-R1195790...,https://static.ah.nl/static/recepten/img_RAM_P...,5,2


In [3]:
usersdf = pd.read_csv("users.csv", index_col=0) 
# recipesdf = pd.read_csv("recipes_kaggle.csv")
# reviewsdf = pd.read_csv("reviews_kaggle.csv")
#recipesdf.head()
# reviewsdf = reviewsdf.head(10000)


In [4]:
def recepten_bij_ingredient(ingredient):
    a = ingredientsdf.loc[ingredientsdf['ingredient'] == ingredient].recipe.to_list()
    return a
def ingredient_bij_recept(gerecht):
    b = ingredientsdf.loc[ingredientsdf['recipe'] == gerecht].ingredient.unique().tolist()
    return b
def rating_bij_recept(gerecht):
    c = recipesdf.loc[recipesdf['title'] == gerecht].Rating.unique().tolist()
    return c
def Id_bij_recept(gerecht):
    d = recipesdf.loc[recipesdf['title'] == gerecht].RecipeId.unique().tolist()
    return d
#TAGS
def recepten_bij_tag(tag):
    a = tagsdf.loc[tagsdf['tag'] == tag].recipe.to_list()
    return a

def tags_bij_recept(gerecht):
    b = tagsdf.loc[tagsdf['recipe'] == gerecht].tag.unique()
    return b

In [5]:
diner_recepten = list(set(recepten_bij_tag('diner') + recepten_bij_tag('hoofdgerecht')))

list_of_ingredients = ingredientsdf[ingredientsdf['recipe'].isin(diner_recepten)].ingredient.unique()
list_of_Tags = tagsdf[tagsdf['recipe'].isin(diner_recepten)].tag.unique().tolist()
tagsdf = tagsdf[tagsdf['recipe'].isin(diner_recepten)]

In [6]:
def createSimulatedUserRecept(randomTag = random.choices(list_of_Tags, k=1000), K=10, U=1000):
    recepten =  []
    realer = []
    for i in range(U):
        recepten.append(random.choices(recepten_bij_tag(randomTag[i]),k=K))
    for i in recepten:
        for j in i:
            realer.append(j)
    return realer

def createUserId(U):
    userId = []
    for i in range(U):
        for j in range(10):
            userId.append(i)
    return userId

def allRatings(lijstje):
    ratings = []
    for i in range(10000):
        ratings.append(rating_bij_recept(lijstje[i]))
    return ratings

def allRecipeId(lijstje):
    Id = []
    for i in range(10000):
        Id.append(Id_bij_recept(lijstje[i]))
    return Id

def unlist(lijstje):
    realer = []
    for i in lijstje:
        for j in i:
            realer.append(j)
    return realer

In [7]:
gerechten = createSimulatedUserRecept()
userid = createUserId(1000)

rats = allRatings(gerechten)
ratings = unlist(rats)

Id = allRecipeId(gerechten)
recip = unlist(Id)


Usersesdf = pd.DataFrame()
Usersesdf['userId'] = userid
Usersesdf['recept'] = gerechten
Usersesdf['rating'] = ratings
Usersesdf['recipeId'] = recip
Usersesdf.set_index('userId')
Usersesdf.drop_duplicates()
Usersesdf.to_csv('users.csv')
display(Usersesdf)

,userId,recept,rating,recipeId
0,0,Zalmvissticks uit de oven met taco's en salsa,1,133
1,0,Wraps met sprinkhanen,2,4833
2,0,Zalmvissticks uit de oven met taco's en salsa,1,133
3,0,Rijkgevulde wraps met kidneybonen,4,3150
4,0,Gegrilde wraps met tomaten-mangosalade,5,1799
...,...,...,...,...
9995,999,Fish &amp; chips met spitskoolsla,4,6397
9996,999,Yorkhamquiches,4,8042
9997,999,Jamie Oliver's bonen op toast,4,6059
9998,999,IJssmoothie aardbei-framboos,2,6477
